In [9]:
!unzip '/content/drive/My Drive/ML_Artivatic_dataset.zip'

Archive:  /content/drive/My Drive/ML_Artivatic_dataset.zip
   creating: ML_Artivatic_dataset/
  inflating: ML_Artivatic_dataset/sample_submission.csv  
  inflating: ML_Artivatic_dataset/test_indessa.csv  
  inflating: ML_Artivatic_dataset/train_indessa.csv  


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
train=pd.read_csv('/content/ML_Artivatic_dataset/train_indessa.csv')

In [12]:
test=pd.read_csv('/content/ML_Artivatic_dataset/test_indessa.csv')

In [13]:
train.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,batch_enrolled,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,verification_status_joint,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,58189336,14350,14350,14350.0,36 months,,19.19,E,E3,clerk,9 years,OWN,28700.0,Source Verified,n,NaN,debt_consolidation,Debt consolidation,349xx,FL,33.88,0.0,1.0,50.0,75.0,14.0,1.0,22515.0,73.1,28.0,f,1173.84,0.0,0.0,0.0,0.0,74.0,INDIVIDUAL,NaN,26th week,0.0,0.0,28699.0,30800.0,0
1,70011223,4800,4800,4800.0,36 months,BAT1586599,10.99,B,B4,Human Resources Specialist,< 1 year,MORTGAGE,65000.0,Source Verified,n,NaN,home_improvement,Home improvement,209xx,MD,3.64,0.0,1.0,NaN,NaN,6.0,0.0,7624.0,23.2,13.0,w,83.95,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,9th week,0.0,0.0,9974.0,32900.0,0
2,70255675,10000,10000,10000.0,36 months,BAT1586599,7.26,A,A4,Driver,2 years,OWN,45000.0,Not Verified,n,NaN,debt_consolidation,Debt consolidation,447xx,OH,18.42,0.0,0.0,NaN,NaN,5.0,0.0,10877.0,31.2,19.0,w,56.47,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,9th week,0.0,65.0,38295.0,34900.0,0
3,1893936,15000,15000,15000.0,36 months,BAT4808022,19.72,D,D5,Us office of Personnel Management,10+ years,RENT,105000.0,Not Verified,n,> My goal is to obtain a loan to pay off my hi...,debt_consolidation,Debt consolidation,221xx,VA,14.97,0.0,2.0,46.0,NaN,10.0,0.0,13712.0,55.5,21.0,f,4858.62,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,135th week,0.0,0.0,55564.0,24700.0,0
4,7652106,16000,16000,16000.0,36 months,BAT2833642,10.64,B,B2,LAUSD-HOLLYWOOD HIGH SCHOOL,10+ years,RENT,52000.0,Verified,n,NaN,credit_card,refi,900xx,CA,20.16,0.0,0.0,NaN,NaN,11.0,0.0,35835.0,76.2,27.0,w,2296.41,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,96th week,0.0,0.0,47159.0,47033.0,0


In [14]:
train['source']='train'
test['source']='test'

In [15]:
train.shape,test.shape

((532428, 46), (354951, 45))

In [16]:
combined=pd.concat([train,test],ignore_index=True)

In [17]:
df=combined.copy()

In [18]:
def missing(df):
    missing_number=df.isnull().sum()
    missing_percent=100*df.isnull().sum()/len(df)
    missing_table=pd.concat([missing_number,missing_percent],axis=1) 
    missing_table=missing_table.rename(columns={0:'missing_value',1:'percent_missing'})
    missing_table=missing_table[missing_table.iloc[:,1]!=0].sort_values('percent_missing',ascending=False)
    return missing_table

In [19]:
missing(df)

,missing_value,percent_missing
verification_status_joint,886868,99.942415
desc,761599,85.825673
mths_since_last_record,750326,84.555303
mths_since_last_major_derog,665676,75.015974
mths_since_last_delinq,454312,51.197065
loan_status,354951,39.999932
batch_enrolled,130748,14.734178
tot_coll_amt,70276,7.919502
tot_cur_bal,70276,7.919502
total_rev_hi_lim,70276,7.919502


In [20]:
df['diff_funds']=df.funded_amnt-df.funded_amnt_inv

In [21]:
df['diff_loan_funds']=df.loan_amnt-(df.funded_amnt+df.funded_amnt_inv)/2

In [22]:
df.batch_enrolled[~df.batch_enrolled.isna()]=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
df.batch_enrolled[df.batch_enrolled.isna()]=0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
df.batch_enrolled.value_counts()

1    756631
0    130748
Name: batch_enrolled, dtype: int64

In [25]:
df.rename(columns={'batch_enrolled':'is_enrolled'},inplace=True)

In [26]:
df.emp_title.value_counts()

Teacher                            13807
Manager                            11638
Registered Nurse                    5691
Owner                               5512
RN                                  5491
                                   ...  
Microsystems, Inc                      1
logisticize llc                        1
Head of Industry                       1
VP,  Banking Center Manager III        1
Registered Client Assoc                1
Name: emp_title, Length: 289206, dtype: int64

In [27]:
df.drop(columns=['verification_status_joint','desc','mths_since_last_record','mths_since_last_major_derog','emp_title'],
        inplace=True)

In [28]:
df.mths_since_last_delinq[~df.mths_since_last_delinq.isna()]=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
df.mths_since_last_delinq[df.mths_since_last_delinq.isna()]=0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
df.mths_since_last_delinq.value_counts()

0.0    454312
1.0    433067
Name: mths_since_last_delinq, dtype: int64

In [31]:
df.rename(columns={'mths_since_last_delinq':'mths_since_last_delinq_missing'},inplace=True)

In [32]:
def impute(df):
    for col in df.columns:
        if(df[col].isnull().sum()>0):
            if(df[col].dtype=='object'):
                df[col].fillna(df[col].mode()[0],inplace=True)
            else:
                df[col].fillna(df[col].mean(),inplace=True)

In [33]:
impute(df)

In [34]:
df.isnull().sum()

member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
is_enrolled                       0
int_rate                          0
grade                             0
sub_grade                         0
emp_length                        0
home_ownership                    0
annual_inc                        0
verification_status               0
pymnt_plan                        0
purpose                           0
title                             0
zip_code                          0
addr_state                        0
dti                               0
delinq_2yrs                       0
inq_last_6mths                    0
mths_since_last_delinq_missing    0
open_acc                          0
pub_rec                           0
revol_bal                         0
revol_util                        0
total_acc                         0
initial_list_status         

In [35]:
df.corr()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq_missing,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status,diff_funds,diff_loan_funds
member_id,1.000000,0.064461,0.066754,0.072308,-0.142205,0.034146,0.067824,0.049902,-0.127252,0.069499,0.088835,0.080785,0.036416,-0.054854,0.031691,-0.461098,-0.076388,-0.114473,-0.072818,0.051437,0.016961,0.003910,0.007286,0.044331,-0.379489,-0.089949,-0.098868
loan_amnt,0.064461,1.000000,0.999263,0.997115,0.145023,0.332697,0.020675,-0.000159,-0.034167,-0.015733,0.198900,-0.081175,0.333580,0.119517,0.222633,0.533916,0.031395,0.073150,0.052205,-0.016648,0.003947,-0.004108,0.315930,0.300226,-0.074130,0.011179,0.034028
funded_amnt,0.066754,0.999263,1.000000,0.998025,0.145160,0.332466,0.021075,0.000151,-0.034710,-0.015207,0.199382,-0.080834,0.333435,0.120012,0.222568,0.532864,0.031075,0.072687,0.051955,-0.016489,0.004051,-0.004110,0.316137,0.300423,-0.075566,0.008416,0.003624
funded_amnt_inv,0.072308,0.997115,0.998025,1.000000,0.145205,0.331419,0.022185,0.000648,-0.038878,-0.015766,0.199996,-0.079787,0.331736,0.120708,0.222783,0.530382,0.027961,0.070751,0.049392,-0.016060,0.004245,-0.004102,0.315577,0.299903,-0.078906,-0.054415,-0.038226
int_rate,-0.142205,0.145023,0.145160,0.145205,1.000000,-0.072785,0.079903,0.055177,0.227647,0.079790,-0.010381,0.052155,-0.035708,0.269052,-0.038618,0.445679,0.057150,0.106840,0.070867,0.013335,0.026478,0.001090,-0.088222,-0.160330,0.002005,-0.004057,-0.003294
annual_inc,0.034146,0.332697,0.332466,0.331419,-0.072785,1.000000,-0.087409,0.047469,0.035193,0.053346,0.138390,-0.008341,0.295783,0.036713,0.187332,0.128535,0.013500,0.007717,0.006630,-0.003936,0.014775,0.001062,0.405282,0.260473,-0.006710,0.009009,0.014283
dti,0.067824,0.020675,0.021075,0.022185,0.079903,-0.087409,1.000000,-0.002831,-0.008106,-0.012618,0.144249,-0.022925,0.067277,0.087836,0.108055,0.008380,-0.005748,0.001162,0.002159,-0.000292,0.003205,-0.002066,-0.007218,0.032651,-0.050540,-0.018162,-0.018628
delinq_2yrs,0.049902,-0.000159,0.000151,0.000648,0.055177,0.047469,-0.002831,1.000000,0.021375,0.361819,0.051189,-0.011004,-0.032486,-0.017250,0.122713,0.001665,0.017423,-0.000729,0.000989,0.063006,0.130326,0.000204,0.063706,-0.040400,-0.035321,-0.007906,-0.010872
inq_last_6mths,-0.127252,-0.034167,-0.034710,-0.038878,0.227647,0.035193,-0.008106,0.021375,1.000000,0.034441,0.111235,0.056870,-0.017556,-0.087749,0.135584,0.085584,0.031620,0.045586,0.033992,0.006843,-0.005329,0.002921,0.031633,0.005440,0.067553,0.067148,0.051238
mths_since_last_delinq_missing,0.069499,-0.015733,-0.015207,-0.015766,0.079790,0.053346,-0.012618,0.361819,0.034441,1.000000,0.050455,0.033448,-0.073275,-0.020101,0.175321,-0.005806,0.014889,-0.004276,-0.000759,0.050846,0.065564,0.013189,0.053983,-0.085092,-0.038011,0.009255,-0.004764


In [36]:
df.drop(columns=['funded_amnt','funded_amnt_inv','total_rev_hi_lim'],inplace=True)

In [37]:
df.shape

(887379, 40)

In [38]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [39]:
df.drop(columns=['last_week_pay', 'title', 'zip_code', 'addr_state','sub_grade'],inplace=True)

In [40]:
df.is_enrolled=df.is_enrolled.astype(str).astype(int)

In [41]:
df.select_dtypes(include='object').columns

Index(['term', 'grade', 'emp_length', 'home_ownership', 'verification_status',
       'pymnt_plan', 'purpose', 'initial_list_status', 'application_type',
       'source'],
      dtype='object')

In [42]:
encod_col=['term', 'grade', 'emp_length', 'home_ownership', 'verification_status',
       'pymnt_plan', 'purpose', 'initial_list_status', 'application_type']

In [43]:
for col in encod_col:
    df[col]=le.fit_transform(df[col])

In [44]:
df=pd.get_dummies(df,columns=['term', 'grade', 'emp_length', 'home_ownership', 'verification_status',
       'pymnt_plan', 'purpose', 'initial_list_status', 'application_type'],drop_first=True)

In [45]:
df.shape

(887379, 66)

In [46]:
train_df,test_df=df[df.source=='train'],df[df.source=='test']

In [47]:
test_df.drop(columns=['source','loan_status'],inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [48]:
train_df.drop(columns=['source','member_id'],inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [49]:
from sklearn.model_selection import train_test_split

In [50]:
x,y=train_df.loc[:,train_df.columns!='loan_status'],train_df.loc[:,'loan_status']

In [51]:
x.shape,y.shape

((532428, 63), (532428,))

In [52]:
x_train,x_val,y_train,y_val=train_test_split(x,y, test_size = 0.05, random_state = 42)

In [53]:
print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)

(505806, 63) (505806,) (26622, 63) (26622,)


In [54]:
for col in x.columns:
    if(x[col].dtype=='object'):
        print(col)

In [55]:
from xgboost.sklearn import XGBClassifier
import xgboost as xgb   
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.metrics import roc_auc_score

In [76]:
def model_train(clf,x_train,y_train,x_val,y_val):
    xgbtrain=xgb.DMatrix(x_train.values, label=y_train.values)
    xgbparams=clf.get_xgb_params()
    cv=xgb.cv(xgbparams,xgbtrain,num_boost_round=600,nfold=5,metrics='auc',early_stopping_rounds=30,seed=123)
    clf.set_params(n_estimators=cv.shape[0])
    clf.fit(x_train,y_train)
    pred=clf.predict(x_val)
    print(cv.head())
    print(roc_auc_score(y_val,pred))
    print(cv.shape[0])

In [77]:
xgb1 = XGBClassifier(learning_rate =0.1,n_estimators=1000,max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,
                      colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,seed=27,
                      tree_method='gpu_hist')

In [78]:
model_train(xgb1,x_train,y_train,x_val,y_val)

   train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0        0.746451       0.000560       0.745709      0.001613
1        0.753496       0.000444       0.752699      0.001685
2        0.757437       0.000422       0.756506      0.001624
3        0.765770       0.000647       0.764787      0.001721
4        0.766603       0.000488       0.765534      0.001405
0.7586396510670143
600


In [91]:
xgb2=XGBClassifier(learning_rate =0.5,n_estimators=800,max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,
                      colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,seed=27,
                      tree_method='gpu_hist')

In [92]:
model_train(xgb2,x_train,y_train,x_val,y_val)

   train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0        0.746451       0.000560       0.745709      0.001613
1        0.762216       0.002363       0.761445      0.001693
2        0.771063       0.001842       0.769917      0.001283
3        0.777009       0.001643       0.775639      0.001445
4        0.780923       0.001367       0.779358      0.001334
0.7796991423240631
290


In [103]:
new_test=test_df.copy()

In [104]:
new_test.drop('member_id',axis=1,inplace=True)

In [106]:
xgb2.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.6, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=0.8, tree_method='gpu_hist', verbosity=1)

In [107]:
pred_prob=xgb2.predict_proba(new_test)[:,1]

In [112]:
pd.DataFrame({'member_id':test_df.member_id,'loan_status':pred_prob}).set_index('member_id').to_csv('submission.csv',header=False)